In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_stack(x)
        return logits
    
model = NeuralNetwork()

In [10]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.023945  [   64/60000]
loss: 1.061922  [ 6464/60000]
loss: 0.867719  [12864/60000]
loss: 1.009779  [19264/60000]
loss: 0.885239  [25664/60000]
loss: 0.920795  [32064/60000]
loss: 0.958759  [38464/60000]
loss: 0.913703  [44864/60000]
loss: 0.954955  [51264/60000]
loss: 0.894260  [57664/60000]
Test Error: 
 Accuracy: 67.1%, Avg loss: 0.903913 

Epoch 2
-------------------------------
loss: 0.936511  [   64/60000]
loss: 0.995228  [ 6464/60000]
loss: 0.787457  [12864/60000]
loss: 0.946988  [19264/60000]
loss: 0.828013  [25664/60000]
loss: 0.854817  [32064/60000]
loss: 0.905037  [38464/60000]
loss: 0.864380  [44864/60000]
loss: 0.898829  [51264/60000]
loss: 0.847019  [57664/60000]
Test Error: 
 Accuracy: 68.1%, Avg loss: 0.853004 

Epoch 3
-------------------------------
loss: 0.871207  [   64/60000]
loss: 0.946089  [ 6464/60000]
loss: 0.727477  [12864/60000]
loss: 0.900354  [19264/60000]
loss: 0.787074  [25664/60000]
loss: 0.805336  [32064/600